In [1]:
import torch
from unet import UNet

model = UNet(
    dim=64   # dimension that will get multiplied by dim_mults
)

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, model_type, path1, path2):
        internal_image_path_1 = os.path.join(r'Scene01\15-deg-left\frames', path1.split('_')[-1], 'Camera_0')
        internal_image_path_2 = os.path.join(r'Scene01\15-deg-left\frames', path2.split('_')[-1], 'Camera_0')
        self.path1 = os.path.join(path1, internal_image_path_1)
        self.path2 = os.path.join(path2, internal_image_path_2)
        
        file_map1 = {}
        for f in os.listdir(self.path1):
            try:
                index = int(f.split('_')[1].split('.')[0])
            except:
                print(f)
            file_map1[index] = (os.path.join(self.path1, f))
        self.file_map1 = file_map1

        file_map2 = {}
        for f in os.listdir(self.path2):
            try:
                index = int(f.split('_')[1].split('.')[0])
            except:
                print(f)
            file_map2[index] = (os.path.join(self.path2, f))
        self.file_map2 = file_map2

        midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
        if model_type == 'DPT_large':
            self.transform = midas_transforms.dpt_transform 
        elif model_type == 'MiDaS_small':
            self.transform = midas_transforms.small_transform 

    def __getitem__(self, i):
        # rgb input
        img1 = cv2.imread(self.file_map1[i])
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
        rgb = self.transform(img1) # already has batch dimension

        img2 = Image.open(self.file_map2[i])
        convert_tensor = v2.Compose(
            [
                v2.Resize(rgb.shape[-2:]),
                v2.ToImageTensor()
            ]

        )
        depth = convert_tensor(img2)[None,:,:,:]

        return rgb, depth

    def __len__(self):
        return len(os.listdir(self.path1))

depth = r'D:\scalable_analytics_institute\semantic_depth_transformer\data\virtual_kitti\vkitti_2.0.3_depth'
rgb = r'D:\scalable_analytics_institute\semantic_depth_transformer\data\virtual_kitti\vkitti_2.0.3_rgb'
semantic = r'D:\scalable_analytics_institute\semantic_depth_transformer\data\virtual_kitti\vkitti_2.0.3_classSegmentation'